# (4)집계 함수를 groupby와 함께 사용하기

In [2]:
import pandas as pd
nls97 = pd.read_csv('data/nls97b.csv')
nls97.set_index('personid', inplace=True)

In [4]:
# 데이터 구조 검토
nls97.iloc[:, 0:7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8984 entries, 100061 to 999963
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 8984 non-null   object 
 1   birthmonth             8984 non-null   int64  
 2   birthyear              8984 non-null   int64  
 3   highestgradecompleted  6663 non-null   float64
 4   maritalstatus          6672 non-null   object 
 5   childathome            4791 non-null   float64
 6   childnotathome         4791 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 561.5+ KB


In [6]:
# 범주형 데이터 검토
catvars = ['gender', 'maritalstatus', 'highestdegree']

for col in catvars:
    print(col, nls97[col].value_counts().sort_index(), sep='\n\n', end='\n\n\n')

gender

Female    4385
Male      4599
Name: gender, dtype: int64


maritalstatus

Divorced          663
Married          3066
Never-married    2766
Separated         154
Widowed            23
Name: maritalstatus, dtype: int64


highestdegree

0. None             953
1. GED             1146
2. High School     3667
3. Associates       737
4. Bachelors       1673
5. Masters          603
6. PhD               54
7. Professional     120
Name: highestdegree, dtype: int64




In [7]:
# 기술통계 검토
contvars = ['satmath', 'satverbal', 'weeksworked06', 'gpaoverall', 'childathome']

nls97[contvars].describe()

,satmath,satverbal,weeksworked06,gpaoverall,childathome
count,1407.000000,1406.000000,8340.000000,6004.000000,4791.000000
mean,500.590618,499.724040,38.429976,2.818408,1.859320
std,114.953309,112.166256,18.921281,0.616357,1.259053
min,7.000000,14.000000,0.000000,0.100000,0.000000
25%,430.000000,430.000000,27.000000,2.430000,1.000000
50%,500.000000,500.000000,51.000000,2.860000,2.000000
75%,580.000000,570.000000,52.000000,3.260000,3.000000
max,800.000000,800.000000,52.000000,4.170000,9.000000


### 성별에 따른 SAT 수학 점수 확인

In [8]:
nls97.groupby('gender')['satmath'].mean()

gender
Female    486.647757
Male      516.875193
Name: satmath, dtype: float64

### 성별과 학력에 따른 SAT 수학 점수 확인

In [9]:
nls97.groupby(['gender', 'highestdegree'])['satmath'].mean()

gender  highestdegree  
Female  0. None            332.600000
        1. GED             405.000000
        2. High School     430.769231
        3. Associates      458.032787
        4. Bachelors       501.945513
        5. Masters         508.271523
        6. PhD             575.454545
        7. Professional    599.411765
Male    0. None            540.000000
        1. GED             320.000000
        2. High School     467.740586
        3. Associates      481.111111
        4. Bachelors       542.163793
        5. Masters         574.444444
        6. PhD             621.428571
        7. Professional    587.727273
Name: satmath, dtype: float64

### 성별 및 학력별 SAT 수학과 언어 점수 확인

In [10]:
nls97.groupby(['gender', 'highestdegree'])[['satmath', 'satverbal']].mean()

satmath   satverbal
gender highestdegree                          
Female 0. None          332.600000  408.800000
       1. GED           405.000000  390.000000
       2. High School   430.769231  444.314917
       3. Associates    458.032787  466.229508
       4. Bachelors     501.945513  506.294872
       5. Masters       508.271523  533.927152
       6. PhD           575.454545  558.181818
       7. Professional  599.411765  587.058824
Male   0. None          540.000000  483.333333
       1. GED           320.000000  360.000000
       2. High School   467.740586  457.196653
       3. Associates    481.111111  462.444444
       4. Bachelors     542.163793  527.844828
       5. Masters       574.444444  545.222222
       6. PhD           621.428571  622.857143
       7. Professional  587.727273  591.818182

### 건수, 평균, 최댓값, 표준편차 열 추가
- agg함수를 사용해 요약통계 반환

In [12]:
nls97.groupby(['gender', 'highestdegree'])['gpaoverall'].agg(['count', 'mean', 'max', 'std'])

count      mean   max       std
gender highestdegree                                   
Female 0. None            148  2.456419  4.00  0.669446
       1. GED             227  2.310132  3.91  0.658090
       2. High School    1212  2.774860  4.17  0.530336
       3. Associates      290  2.943483  4.00  0.495803
       4. Bachelors       734  3.238529  4.07  0.466571
       5. Masters         312  3.296186  4.08  0.432133
       6. PhD              22  3.460000  4.00  0.451885
       7. Professional     53  3.537736  4.11  0.413548
Male   0. None            193  2.218342  4.00  0.643481
       1. GED             345  2.242145  4.00  0.631721
       2. High School    1436  2.632806  4.00  0.497803
       3. Associates      236  2.727542  3.83  0.488499
       4. Bachelors       560  3.110446  4.10  0.486659
       5. Masters         170  3.304529  4.04  0.447173
       6. PhD              20  3.389000  3.99  0.551895
       7. Professional     38  3.442105  4.00  0.339301

### 딕셔너리를 사용해 복잡한 집계 수행

In [14]:
# Pandas가 데이터프레임 등을 출력할 때 부동 소수점 숫자의 형식이 변경
# 소수점 아래 1자리까지 표시하고, 천 단위로 쉼표를 사용하여 천 단위 구분 기호
pd.options.display.float_format = '{:,.1f}'.format

In [15]:
aggdict = {'weeksworked06':['count', 'mean', 'max', 'std'], 'childathome':['count', 'mean', 'max', 'std']}

In [16]:
nls97.groupby(['highestdegree']).agg(aggdict)

weeksworked06                childathome             
                        count mean  max  std       count mean max std
highestdegree                                                        
0. None                   703 29.7 52.0 21.6         439  1.8 8.0 1.6
1. GED                   1104 33.2 52.0 20.6         693  1.7 9.0 1.5
2. High School           3368 39.4 52.0 18.6        1961  1.9 7.0 1.3
3. Associates             722 40.7 52.0 17.7         428  2.0 6.0 1.1
4. Bachelors             1642 42.2 52.0 16.1         827  1.9 8.0 1.0
5. Masters                601 42.2 52.0 16.1         333  1.9 5.0 0.9
6. PhD                     53 38.2 52.0 18.6          32  2.1 6.0 1.1
7. Professional           117 27.1 52.0 20.4          57  1.8 4.0 0.8

In [17]:
nls97.groupby(['maritalstatus']).agg(aggdict)

weeksworked06                childathome             
                      count mean  max  std       count mean max std
maritalstatus                                                      
Divorced                660 37.5 52.0 19.1         524  1.5 5.0 1.2
Married                3033 40.3 52.0 17.9        2563  2.1 8.0 1.1
Never-married          2734 37.2 52.0 19.1        1502  1.6 9.0 1.3
Separated               153 33.8 52.0 20.2         137  1.5 8.0 1.4
Widowed                  23 37.1 52.0 19.3          18  1.8 5.0 1.4